<a href="https://colab.research.google.com/github/ericaburdett/DS-Unit-2-Applied-Modeling/blob/master/module3-permutation-boostingxgb_eli5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import xgboost as xgb

!pip install eli5

from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
# From https://www.kaggle.com/nikdavis/steam-store-games#steam.csv

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/ericaburdett/games-data/master/steam.csv')

In [0]:
df = df[['appid', 'name']]

In [0]:
# Also from https://www.kaggle.com/nikdavis/steam-store-games#steamspy_tag_data.csv

tags = pd.read_csv('https://raw.githubusercontent.com/ericaburdett/games-data/master/steamspy_tag_data.csv')

In [0]:
games = df.merge(tags, how='inner')

In [7]:
games.shape

(27075, 373)

In [8]:
games.head()

,appid,name,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,4_player_local,4x,6dof,atv,abstract,action,action_rpg,action_adventure,addictive,adventure,agriculture,aliens,alternate_history,america,animation_&_modeling,anime,arcade,arena_shooter,artificial_intelligence,assassin,asynchronous_multiplayer,atmospheric,audio_production,bmx,base_building,baseball,based_on_a_novel,basketball,batman,...,touch_friendly,tower_defense,trackir,trading,trading_card_game,trains,transhumanism,turn_based,turn_based_combat,turn_based_strategy,turn_based_tactics,tutorial,twin_stick_shooter,typing,underground,underwater,unforgiving,utilities,vr,vr_only,vampire,video_production,villain_protagonist,violent,visual_novel,voice_control,voxel,walking_simulator,war,wargame,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,Counter-Strike,144,564,0,0,0,0,0,0,0,0,0,0,0,0,2681,0,0,0,0,0,0,0,0,0,0,0,0,0,151,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,550
1,20,Team Fortress Classic,0,71,0,0,0,0,0,0,0,0,0,0,0,0,208,0,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,0,0,0,0,0,0,0,0,0,0,0,0,0,99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80,0,0,0,0,0,0,5,122,0,0,0
3,40,Deathmatch Classic,0,0,0,0,0,0,0,0,0,0,0,0,0,0,85,0,0,0,0,0,0,0,0,0,0,0,22,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,77,0,0,0,0,0,0,0,0,0,0,0,0,211,0,0,0,87,0,122,0,0,0,0,0,0,0,0,0,73,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# From https://www.kaggle.com/rush4ratio/video-game-sales-with-ratings

ratings = pd.read_csv('https://raw.githubusercontent.com/ericaburdett/games-data/master/Video_Games_Sales_as_at_22_Dec_2016.csv')

In [10]:
ratings.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
ratings = ratings[['Name', 'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count']]

In [0]:
games = games.merge(ratings, left_on='name', right_on='Name', how='inner')

In [13]:
games.dtypes

appid             int64
name             object
1980s             int64
1990s             int64
2.5d              int64
                 ...   
Name             object
Critic_Score    float64
Critic_Count    float64
User_Score       object
User_Count      float64
Length: 378, dtype: object

In [0]:
games = games.dropna()

In [15]:
games.shape

(696, 378)

In [0]:
def str_to_float(string):
  fl = float(string)
  return fl

In [0]:
games['User_Score'] = games['User_Score'].apply(str_to_float)

In [18]:
games.dtypes

appid             int64
name             object
1980s             int64
1990s             int64
2.5d              int64
                 ...   
Name             object
Critic_Score    float64
Critic_Count    float64
User_Score      float64
User_Count      float64
Length: 378, dtype: object

In [0]:
target = 'User_Score'
to_drop = ['appid', 'name', 'Name', 'Critic_Score', 'Critic_Count', 'User_Count']



from sklearn.model_selection import train_test_split

train, test = train_test_split(games, train_size=0.8, random_state=42)
test, val = train_test_split(test, train_size=0.5, random_state=42)

features = train.columns.drop([target] + to_drop)

In [0]:

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import f_regression, SelectKBest

from IPython.display import display, HTML
from ipywidgets import interact
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge


In [22]:
#Baseline
guess = games['User_Score'].mean()
errors = guess - games['User_Score']

mae = errors.abs().mean()
mae

1.0422434436517376

In [28]:
pipeline = make_pipeline(
    RandomForestRegressor()
)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean absolute error: {mae:,.2f}')

Mean absolute error: 0.66


In [32]:
import eli5
from eli5.sklearn import PermutationImportance

model = RandomForestRegressor()
model.fit(X_train, y_train)

permuter = PermutationImportance(
    model,
    n_iter=5,
    random_state=42
)

permuter.fit(X_val, y_val)



PermutationImportance(cv='prefit',
                      estimator=RandomForestRegressor(bootstrap=True,
                                                      ccp_alpha=0.0,
                                                      criterion='mse',
                                                      max_depth=None,
                                                      max_features='auto',
                                                      max_leaf_nodes=None,
                                                      max_samples=None,
                                                      min_impurity_decrease=0.0,
                                                      min_impurity_split=None,
                                                      min_samples_leaf=1,
                                                      min_samples_split=2,
                                                      min_weight_fraction_leaf=0.0,
                                                      n_estimators=100

In [37]:
feature_names = X_val.columns.tolist()
pd.Series(permuter.feature_importances_, feature_names).sort_values(ascending=False)

classic             0.063872
simulation          0.062473
co_op               0.062395
great_soundtrack    0.053943
singleplayer        0.034969
                      ...   
platformer         -0.014911
adventure          -0.015314
comedy             -0.025759
strategy           -0.044523
3d_platformer      -0.067042
Length: 371, dtype: float64

In [38]:
eli5.show_weights(
    permuter,
    top=None,
    feature_names=feature_names
)

Weight,Feature
0.0639 ± 0.1061,classic
0.0625 ± 0.0282,simulation
0.0624 ± 0.0297,co_op
0.0539 ± 0.1231,great_soundtrack
0.0350 ± 0.0548,singleplayer
0.0310 ± 0.0570,atmospheric
0.0286 ± 0.0068,open_world
0.0277 ± 0.0385,2.5d
0.0237 ± 0.0310,action
0.0218 ± 0.0331,sports


In [39]:
print('Shape before removing features', X_train.shape)

Shape before removing features (556, 371)


In [0]:
minimum_importance = 0

mask = permuter.feature_importances_ > minimum_importance
features = X_train.columns[mask]

X_train = X_train[features]

In [43]:
X_val = X_val[features]
X_train = X_train[features]
X_test = X_test[features]

model = RandomForestRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean absolute error: {mae:,.2f}')

Mean absolute error: 0.71


# XGBoost


In [44]:
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean absolute error: {mae:,.2f}')

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[13:03:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean absolute error: 0.78
